In [2]:
import pandas as pd
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.linear_model import LogisticRegression

# Функция для загрузки данных
def load_data(sample_size):
    attributes = pd.read_parquet('F:/competition/attributes.parquet')
    resnet = pd.read_parquet('F:/competition/resnet.parquet')
    text_and_bert = pd.read_parquet('F:/competition/text_and_bert.parquet')
    train = pd.read_parquet('F:/competition/train.parquet')
    test = pd.read_parquet('F:/competition/test.parquet')

    if sample_size < 1:
        attributes = attributes.sample(frac=sample_size, random_state=42)
        resnet = resnet.sample(frac=sample_size, random_state=42)
        text_and_bert = text_and_bert.sample(frac=sample_size, random_state=42)
        train = train.sample(frac=sample_size, random_state=42)
        test = test.sample(frac=sample_size, random_state=42)

    return attributes, resnet, text_and_bert, train, test

# Загрузка данных
attributes, resnet, text_and_bert, train, test = load_data(sample_size=1)

In [3]:
attributes

,variantid,categories,characteristic_attributes_mapping
0,47920382,"{""1"": ""EPG"", ""2"": ""Детские товары"", ""3"": ""Игру...","{""Цвет товара"": [""бежевый"", ""светло-розовый""],..."
1,49801845,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Количество в упаковке, шт"": [""1""], ""Бренд"": ..."
2,49853444,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Бренд"": [""Vervaco""], ""Тип"": [""Набор для выши..."
3,49893028,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Цвет товара"": [""серый""], ""Ширина, см"": [""0.8..."
4,49987483,"{""1"": ""EPG"", ""2"": ""Хобби и творчество"", ""3"": ""...","{""Цвет товара"": [""разноцветный""], ""Название цв..."
...,...,...,...
2252564,1799186009,"{""1"": ""EPG"", ""2"": ""Спорт и отдых"", ""3"": ""Скейт...","{""Материал деки"": [""ABS пластик"", ""Алюминий""],..."
2252565,1801305376,"{""1"": ""Книги и цифровые книги"", ""2"": ""Книги"", ...","{""Год выпуска"": [""2024""], ""Количество страниц""..."
2252566,1802944592,"{""1"": ""EPG"", ""2"": ""Спорт и отдых"", ""3"": ""Аксес...","{""Толщина, мм"": [""0.06""], ""Бренд"": [""Daiwa""], ..."
2252567,1803038155,"{""1"": ""Книги и цифровые книги"", ""2"": ""Книги"", ...","{""Год выпуска"": [""2022""], ""Количество страниц""..."


In [4]:
# Инициализация модели и токенизатора
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny2")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

C:\Anaconda\envs\ilyhkino_okrushenie\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(83828, 312, padding_idx=0)
    (position_embeddings): Embedding(2048, 312)
    (token_type_embeddings): Embedding(2, 312)
    (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-2): 3 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=312, out_features=312, bias=True)
            (key): Linear(in_features=312, out_features=312, bias=True)
            (value): Linear(in_features=312, out_features=312, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=312, out_features=312, bias=True)
            (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)

In [5]:
def embed_bert_cls(text, model, tokenizer):
    # Токенизация текста: преобразует текст в формат, пригодный для модели BERT
    # padding=True добавляет нули к коротким последовательностям, truncation=True обрезает длинные последовательности
    # return_tensors="pt" возвращает результат в формате тензоров PyTorch
    t = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    
    # torch.no_grad() отключает автоматическое вычисление градиентов для экономии памяти и ускорения инференса
    with torch.no_grad():
        # Прогоняем данные через модель, перемещая их на устройство (GPU или CPU)
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    
    # Извлекаем эмбеддинги CLS-токена (он расположен в начале последовательности и предназначен для классификации)
    embeddings = model_output.last_hidden_state[:, 0, :]
    
    # Нормализуем эмбеддинги (делаем их единичными векторами), чтобы можно было сравнивать их по косинусному сходству
    embeddings = torch.nn.functional.normalize(embeddings)
    
    # Преобразуем тензор с устройства (GPU или CPU) в numpy-массив на CPU
    return embeddings[0].cpu().numpy()


In [14]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def create_features(row, bert_df, resnet_df):
    variantid1, variantid2 = row['variantid1'], row['variantid2']
    
    # Эмбеддинги названий из BERT
    name_bert_1_values = bert_df.loc[bert_df['variantid'] == variantid1, 'name_bert_64'].values
    name_bert_2_values = bert_df.loc[bert_df['variantid'] == variantid2, 'name_bert_64'].values

    # Проверка наличия данных
    if len(name_bert_1_values) == 0 or len(name_bert_2_values) == 0:
        print(f"Эмбеддинги для variantid1: {variantid1} или variantid2: {variantid2} не найдены.")
        return [0, 0]

    try:
        name_bert_1 = np.array(name_bert_1_values[0]).reshape(1, -1)  # Преобразуем в 2D массив
        name_bert_2 = np.array(name_bert_2_values[0]).reshape(1, -1)  # Преобразуем в 2D массив
    except Exception as e:
        print(f"Ошибка при преобразовании эмбеддингов названий: {e}")
        return [0, 0]

    # Косинусное сходство между эмбеддингами названий
    try:
        name_similarity = cosine_similarity(name_bert_1, name_bert_2)[0][0]
    except Exception as e:
        print(f"Ошибка при вычислении косинусного сходства для названий: {e}")
        name_similarity = 0

    # Эмбеддинги изображений из ResNet
    pic_embedding_1_values = resnet_df.loc[resnet_df['variantid'] == variantid1, 'main_pic_embeddings_resnet_v1'].values
    pic_embedding_2_values = resnet_df.loc[resnet_df['variantid'] == variantid2, 'main_pic_embeddings_resnet_v1'].values

    # Проверка наличия данных
    if len(pic_embedding_1_values) == 0 or len(pic_embedding_2_values) == 0:
        print(f"Эмбеддинги изображений для variantid1: {variantid1} или variantid2: {variantid2} не найдены.")
        return [name_similarity, 0]

    try:
        pic_embedding_1 = np.array(pic_embedding_1_values[0]).reshape(1, -1)  # Преобразуем в 2D массив
        pic_embedding_2 = np.array(pic_embedding_2_values[0]).reshape(1, -1)  # Преобразуем в 2D массив
    except Exception as e:
        print(f"Ошибка при преобразовании эмбеддингов изображений: {e}")
        return [name_similarity, 0]

    # Косинусное сходство между эмбеддингами изображений
    try:
        pic_similarity = cosine_similarity(pic_embedding_1, pic_embedding_2)[0][0]
    except Exception as e:
        print(f"Ошибка при вычислении косинусного сходства для изображений: {e}")
        pic_similarity = 0

    return [name_similarity, pic_similarity]


In [16]:
# Создание обучающего датасета
small_train = train.head(10)
train_features = train.apply(lambda row: create_features(row, text_and_bert, resnet), axis=1)
train_features = np.array(train_features.tolist())
train_labels = train['target'].values

ValueError: setting an array element with a sequence.

In [10]:
# Посмотрите на несколько примеров данных
print(resnet['main_pic_embeddings_resnet_v1'].head())

# Проверьте тип данных
print(type(resnet['main_pic_embeddings_resnet_v1'].iloc[0]))


0    [[0.8170074820518494, 0.9416620135307312, 0.31...
1    [[-0.43339717388153076, -0.17318281531333923, ...
2    [[0.11314830183982849, -0.34010639786720276, -...
3    [[0.25037717819213867, 0.33753663301467896, 0....
4    [[0.43453288078308105, 0.09419603645801544, -0...
Name: main_pic_embeddings_resnet_v1, dtype: object
<class 'numpy.ndarray'>


In [7]:
pic_embedding_1

NameError: name 'pic_embedding_1' is not defined

In [ ]:




# Обучение модели
model = LogisticRegression()
model.fit(train_features, train_labels)

# Применение на тестовом датасете
test_features = test.apply(lambda row: create_features(row, text_and_bert, resnet), axis=1)
test_features = np.array(test_features.tolist())
predictions = model.predict(test_features)

# Сохранение результатов
submission = pd.DataFrame({
    'variantid1': test['variantid1'],
    'variantid2': test['variantid2'],
    'target': predictions
})
submission.to_csv('submission.csv', index=False)


In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, auc
import joblib
import json  # Добавлен импорт модуля json

# Загрузка данных
def load_data(sample_size=0.1):
    attributes = pd.read_parquet('F:/competition/attributes.parquet')
    resnet = pd.read_parquet('F:/competition/resnet.parquet')
    text_and_bert = pd.read_parquet('F:/competition/text_and_bert.parquet')
    train = pd.read_parquet('F:/competition/train.parquet')
    test = pd.read_parquet('F:/competition/test.parquet')

    if sample_size < 1:
        attributes = attributes.sample(frac=sample_size, random_state=42)
        resnet = resnet.sample(frac=sample_size, random_state=42)
        text_and_bert = text_and_bert.sample(frac=sample_size, random_state=42)
        train = train.sample(frac=sample_size, random_state=42)
        test = test.sample(frac=sample_size, random_state=42)

    return attributes, resnet, text_and_bert, train, test

# Загрузка данных
attributes, resnet, text_and_bert, train, test = load_data(sample_size=0.1)

In [25]:
def prepare_features(train, attributes, resnet, text_and_bert):
    # Присоединяем атрибуты
    train = train.merge(attributes[['variantid', 'characteristic_attributes_mapping']], 
                        left_on='variantid1', right_on='variantid', how='left').drop('variantid', axis=1)
    train = train.merge(attributes[['variantid', 'characteristic_attributes_mapping']], 
                        left_on='variantid2', right_on='variantid', how='left', suffixes=('_1', '_2')).drop('variantid', axis=1)

    # Присоединяем текстовые эмбеддинги BERT
    train = train.merge(text_and_bert[['variantid', 'name_bert_64']], 
                        left_on='variantid1', right_on='variantid', how='left').drop('variantid', axis=1)
    train = train.merge(text_and_bert[['variantid', 'name_bert_64']], 
                        left_on='variantid2', right_on='variantid', how='left', suffixes=('_1', '_2')).drop('variantid', axis=1)

    # Преобразование эмбеддингов в числовые массивы
    for col in ['name_bert_64_1', 'name_bert_64_2']:
        # Преобразуем только если значение не NaN
        train[col] = train[col].apply(lambda x: np.array(json.loads(x)) if isinstance(x, str) else np.nan)

    # Векторы ResNet для изображений
    train = train.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], 
                        left_on='variantid1', right_on='variantid', how='left').drop('variantid', axis=1)
    train = train.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], 
                        left_on='variantid2', right_on='variantid', how='left', suffixes=('_resnet1', '_resnet2')).drop('variantid', axis=1)

    # Преобразование эмбеддингов ResNet в числовые массивы
    for col in ['main_pic_embeddings_resnet_v1_resnet1', 'main_pic_embeddings_resnet_v1_resnet2']:
        # Преобразуем только если значение не NaN
        train[col] = train[col].apply(lambda x: np.array(json.loads(x)) if isinstance(x, str) else np.nan)

    # Проверяем создание всех эмбеддингов в виде массива
    if any(col not in train.columns for col in ['name_bert_64_1', 'name_bert_64_2', 'main_pic_embeddings_resnet_v1_resnet1', 'main_pic_embeddings_resnet_v1_resnet2']):
        raise KeyError("Не удалось создать один из столбцов с эмбеддингами.")

    # Объединение всех эмбеддингов в единый вектор признаков
    train['combined_features'] = train.apply(lambda row: np.concatenate([
        row['name_bert_64_1'] if isinstance(row['name_bert_64_1'], np.ndarray) else np.zeros(64), 
        row['name_bert_64_2'] if isinstance(row['name_bert_64_2'], np.ndarray) else np.zeros(64), 
        row['main_pic_embeddings_resnet_v1_resnet1'] if isinstance(row['main_pic_embeddings_resnet_v1_resnet1'], np.ndarray) else np.zeros(2048), 
        row['main_pic_embeddings_resnet_v1_resnet2'] if isinstance(row['main_pic_embeddings_resnet_v1_resnet2'], np.ndarray) else np.zeros(2048)
    ]), axis=1)

    # Проверяем успешное создание столбца combined_features
    if 'combined_features' not in train.columns:
        raise KeyError("Не удалось создать столбец 'combined_features'.")

    return train
# Подготовка признаков
train = prepare_features(train, attributes, resnet, text_and_bert)

In [26]:
print(train.isna().sum())


variantid1                                    0
variantid2                                    0
target                                        0
characteristic_attributes_mapping_1      105253
characteristic_attributes_mapping_2      105228
name_bert_64_1                           116852
name_bert_64_2                           116852
main_pic_embeddings_resnet_v1_resnet1    116852
main_pic_embeddings_resnet_v1_resnet2    116852
combined_features                             0
dtype: int64


In [27]:
train.fillna(0, inplace=True)


In [28]:
train

,variantid1,variantid2,target,characteristic_attributes_mapping_1,characteristic_attributes_mapping_2,name_bert_64_1,name_bert_64_2,main_pic_embeddings_resnet_v1_resnet1,main_pic_embeddings_resnet_v1_resnet2,combined_features
0,938776155,473815138,1,0,0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1349745192,1349703019,0,0,0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1450853415,1450853943,1,0,0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,741508506,413821437,0,0,0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1337038519,1313965970,1,"{""Комплектация"": [""Стайлер, инструкция, упаков...",0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...
116847,101957372,1269490207,0,"{""Артикул"": [""665334""], ""Партномер (артикул пр...",0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
116848,700695167,700666712,0,0,0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
116849,1557204320,1010812448,1,0,0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
116850,1480454634,1491520915,0,0,0,0.0,0.0,0.0,0.0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [29]:
# Разделение на тренировочную и валидационную выборки
X = np.vstack(train['combined_features'].values)
y = train['target'].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Обучение модели
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_scaled, y_train)

# Прогнозирование
y_pred_prob = logreg.predict_proba(X_val_scaled)[:, 1]

# Вычисление метрики PR AUC
precision, recall, _ = precision_recall_curve(y_val, y_pred_prob)
pr_auc = auc(recall, precision)
print(f"PR AUC: {pr_auc}")

# Сохранение модели
joblib.dump(logreg, 'model.pkl')
joblib.dump(scaler, 'scaler.pkl')

PR AUC: 0.7412177484917205


['scaler.pkl']

In [30]:
# Обучение модели CatBoost
catboost_model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, eval_metric='PRAUC', verbose=200)
catboost_model.fit(X_train_scaled, y_train, eval_set=(X_val_scaled, y_val))

# Прогнозирование
y_pred_prob = catboost_model.predict_proba(X_val_scaled)[:, 1]

# Вычисление метрики PR AUC
precision, recall, _ = precision_recall_curve(y_val, y_pred_prob)
pr_auc = auc(recall, precision)
print(f"PR AUC: {pr_auc}")

# Сохранение модели и скейлера
joblib.dump(catboost_model, 'catboost_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

CatBoostError: C:/Go_Agent/pipelines/BuildMaster/catboost.git/catboost/libs/data/quantization.cpp:2420: All features are either constant or ignored.

In [19]:

# Применение модели к тестовым данным
def predict_on_test(test, text_and_bert, resnet, attributes, model, scaler):
    test = prepare_features(test, attributes, resnet, text_and_bert)
    X_test = np.vstack(test['combined_features'].values)
    X_test_scaled = scaler.transform(X_test)
    test['target'] = model.predict_proba(X_test_scaled)[:, 1]
    test[['variantid1', 'variantid2', 'target']].to_csv('submission.csv', index=False)

# Инференс
predict_on_test(test, text_and_bert, resnet, attributes, logreg, scaler)


In [21]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, auc
import joblib
from catboost import CatBoostClassifier  # Добавляем CatBoostClassifier

# Загрузка данных
def load_data(sample_size=0.1):
    attributes = pd.read_parquet('F:/competition/attributes.parquet')
    resnet = pd.read_parquet('F:/competition/resnet.parquet')
    text_and_bert = pd.read_parquet('F:/competition/text_and_bert.parquet')
    train = pd.read_parquet('F:/competition/train.parquet')
    test = pd.read_parquet('F:/competition/test.parquet')

    if sample_size < 1:
        attributes = attributes.sample(frac=sample_size, random_state=42)
        resnet = resnet.sample(frac=sample_size, random_state=42)
        text_and_bert = text_and_bert.sample(frac=sample_size, random_state=42)
        train = train.sample(frac=sample_size, random_state=42)
        test = test.sample(frac=sample_size, random_state=42)

    return attributes, resnet, text_and_bert, train, test

# Загрузка данных
attributes, resnet, text_and_bert, train, test = load_data(sample_size=0.1)

# Преобразование данных
def prepare_features(train, attributes, resnet, text_and_bert):
    # Объединение данных по variantid
    train = train.merge(text_and_bert[['variantid', 'name_bert_64']], left_on='variantid1', right_on='variantid', how='left').drop('variantid', axis=1)
    train = train.merge(text_and_bert[['variantid', 'name_bert_64']], left_on='variantid2', right_on='variantid', suffixes=('_1', '_2')).drop('variantid', axis=1)
    
    # Преобразование эмбеддингов в числовые массивы
    for col in ['name_bert_64_1', 'name_bert_64_2']:
        train[col] = train[col].apply(lambda x: np.array(json.loads(x)))

    # Векторы ResNet для изображений
    train = train.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid1', right_on='variantid', how='left').drop('variantid', axis=1)
    train = train.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid2', right_on='variantid', suffixes=('_img1', '_img2')).drop('variantid', axis=1)
    
    # Преобразование эмбеддингов изображений в числовые массивы
    for col in ['main_pic_embeddings_resnet_v1_img1', 'main_pic_embeddings_resnet_v1_img2']:
        train[col] = train[col].apply(lambda x: np.array(json.loads(x)))

    # Атрибуты
    train = train.merge(attributes[['variantid', 'characteristic_attributes_mapping']], left_on='variantid1', right_on='variantid', how='left').drop('variantid', axis=1)
    train = train.merge(attributes[['variantid', 'characteristic_attributes_mapping']], left_on='variantid2', right_on='variantid', suffixes=('_attr1', '_attr2')).drop('variantid', axis=1)

    # Преобразование атрибутов
    train['attribute_similarity'] = train.apply(lambda row: compute_similarity(row['characteristic_attributes_mapping_attr1'], row['characteristic_attributes_mapping_attr2']), axis=1)

    # Конкатенация всех эмбеддингов и признаков
    train['combined_features'] = train.apply(lambda row: np.concatenate([row['name_bert_64_1'], row['name_bert_64_2'], row['main_pic_embeddings_resnet_v1_img1'], row['main_pic_embeddings_resnet_v1_img2'], [row['attribute_similarity']]]), axis=1)

    return train

# Подготовка признаков
train = prepare_features(train, attributes, resnet, text_and_bert)

# Разделение на тренировочную и валидационную выборки
X = np.vstack(train['combined_features'].values)
y = train['target'].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование данных
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Обучение модели CatBoost
catboost_model = CatBoostClassifier(iterations=1000, learning_rate=0.1, depth=6, eval_metric='PRAUC', verbose=200)
catboost_model.fit(X_train_scaled, y_train, eval_set=(X_val_scaled, y_val))

# Прогнозирование
y_pred_prob = catboost_model.predict_proba(X_val_scaled)[:, 1]

# Вычисление метрики PR AUC
precision, recall, _ = precision_recall_curve(y_val, y_pred_prob)
pr_auc = auc(recall, precision)
print(f"PR AUC: {pr_auc}")

# Сохранение модели и скейлера
joblib.dump(catboost_model, 'catboost_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

# Применение модели к тестовым данным
def predict_on_test(test, text_and_bert, resnet, attributes, model, scaler):
    test = prepare_features(test, attributes, resnet, text_and_bert)
    X_test = np.vstack(test['combined_features'].values)
    X_test_scaled = scaler.transform(X_test)
    test['target'] = model.predict_proba(X_test_scaled)[:, 1]
    test[['variantid1', 'variantid2', 'target']].to_csv('submission.csv', index=False)

# Инференс
predict_on_test(test, text_and_bert, resnet, attributes, catboost_model, scaler)


TypeError: the JSON object must be str, bytes or bytearray, not float

In [ ]:
# Пример с TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Пример создания векторизатора
tfidf_vectorizer = TfidfVectorizer(max_features=3000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_text)
joblib.dump(tfidf_vectorizer, 'vectorizer.pkl')  # Сохранение векторизатора


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc
import joblib
import json

# Путь к данным  
attributes_path = 'F:/competition/attributes.parquet'  
resnet_path = 'F:/competition/resnet.parquet'  
text_and_bert_path = 'F:/competition/text_and_bert.parquet'  
train_path = 'F:/competition/train.parquet'  
test_path = 'F:/competition/test.parquet'  

# Функция для загрузки данных  
def load_data(sample_size=0.1):  
    # Загрузка данных из файлов Parquet  
    attributes = pd.read_parquet(attributes_path)  
    resnet = pd.read_parquet(resnet_path)  
    text_and_bert = pd.read_parquet(text_and_bert_path)  
    train = pd.read_parquet(train_path)  
    test = pd.read_parquet(test_path)  

    # Если sample_size меньше 1, то выборка данных  
    if sample_size < 1:  
        attributes = attributes.sample(frac=sample_size, random_state=42)  
        resnet = resnet.sample(frac=sample_size, random_state=42)  
        text_and_bert = text_and_bert.sample(frac=sample_size, random_state=42)  
        train = train.sample(frac=sample_size, random_state=42)  
        test = test.sample(frac=sample_size, random_state=42)  

    # Возвращаем 5 DataFrame'ов
    return attributes, resnet, text_and_bert, train, test  

# Обработка текстовых данных
def process_text_and_bert(df):
    # Объединение столбцов name и description в единый текст
    df['combined_text'] = df['name'] + ' ' + df['description']
    return df

# Объединение данных
def merge_data(train, resnet, text_and_bert):
    # Объединение с эмбеддингами ResNet
    train_data = train.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid1', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_1'})
    train_data = train_data.drop(columns=['variantid'])

    train_data = train_data.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid2', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_2'})
    train_data = train_data.drop(columns=['variantid'])

    # Объединение с текстовыми данными и эмбеддингами BERT
    train_data = train_data.merge(text_and_bert[['variantid', 'combined_text', 'name_bert_64']], left_on='variantid1', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'combined_text': 'text_1', 'name_bert_64': 'text_embedding_1'})
    train_data = train_data.drop(columns=['variantid'])

    train_data = train_data.merge(text_and_bert[['variantid', 'combined_text', 'name_bert_64']], left_on='variantid2', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'combined_text': 'text_2', 'name_bert_64': 'text_embedding_2'})
    train_data = train_data.drop(columns=['variantid'])

    # Удаление строк, в которых есть пропущенные значения
    train_data = train_data.dropna()

    return train_data

# Объединение эмбеддингов
def combine_embeddings(row):
    # Объединение эмбеддингов изображений и текста
    pic_embeddings = np.concatenate([row['pic_embeddings_1'][0], row['pic_embeddings_2'][0]])
    text_embeddings = np.concatenate([row['text_embedding_1'], row['text_embedding_2']])
    return np.concatenate([pic_embeddings, text_embeddings])

# Подготовка данных для обучения модели
def prepare_data(train_data):
    # Объединение эмбеддингов изображений и текстов
    train_data['combined_embeddings'] = train_data.apply(combine_embeddings, axis=1)

    # Преобразование объединенных эмбеддингов в массив numpy
    X = np.vstack(train_data['combined_embeddings'].values)
    y = train_data['target']

    # Разделение данных на обучающую и валидационную выборки
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=23)

    return X_train, X_val, y_train, y_val

# Обучение модели
def train_model(X_train, y_train):
    model = LogisticRegression(max_iter=2000)
    model.fit(X_train, y_train)
    
    # Сохранение обученной модели
    joblib.dump(model, 'baseline.pkl')
    return model

# Оценка модели
def evaluate_model(model, X_val, y_val):
    y_pred_prob = model.predict_proba(X_val)[:, 1]
    y_pred = (y_pred_prob >= 0.5).astype(int)

    precision, recall, _ = precision_recall_curve(y_val, y_pred_prob)
    prauc = auc(recall, precision)
    print(f'PRAUC: {prauc}')

# Главная функция для запуска всего процесса
def main():
    attributes, resnet, text_and_bert, train, test = load_data()

    # Обработка текстовых данных
    text_and_bert = process_text_and_bert(text_and_bert)

    # Объединение данных
    train_data = merge_data(train, resnet, text_and_bert)

    # Подготовка данных для обучения модели
    X_train, X_val, y_train, y_val = prepare_data(train_data)

    # Обучение модели
    model = train_model(X_train, y_train)
    
    # Оценка модели
    evaluate_model(model, X_val, y_val)

if __name__ == "__main__":
    main()


PRAUC: 0.5572370449850078


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve, auc
import joblib
import json

# Путь к данным  
attributes_path = 'F:/competition/attributes.parquet'  
resnet_path = 'F:/competition/resnet.parquet'  
text_and_bert_path = 'F:/competition/text_and_bert.parquet'  
train_path = 'F:/competition/train.parquet'  
test_path = 'F:/competition/test.parquet'  

# Функция для загрузки данных  
def load_data(sample_size=0.1):  
    # Загрузка данных из файлов Parquet  
    attributes = pd.read_parquet(attributes_path)  
    resnet = pd.read_parquet(resnet_path)  
    text_and_bert = pd.read_parquet(text_and_bert_path)  
    train = pd.read_parquet(train_path)  
    test = pd.read_parquet(test_path)  

    # Если sample_size меньше 1, то выборка данных  
    if sample_size < 1:  
        attributes = attributes.sample(frac=sample_size, random_state=42)  
        resnet = resnet.sample(frac=sample_size, random_state=42)  
        text_and_bert = text_and_bert.sample(frac=sample_size, random_state=42)  
        train = train.sample(frac=sample_size, random_state=42)  
        test = test.sample(frac=sample_size, random_state=42)  

    # Возвращаем 5 DataFrame'ов
    return attributes, resnet, text_and_bert, train, test  

# Обработка текстовых данных
def process_text_and_bert(df):
    # Объединение столбцов name и description в единый текст
    df['combined_text'] = df['name'] + ' ' + df['description']
    return df

# Объединение данных
def merge_data(train, resnet, text_and_bert):
    # Объединение с эмбеддингами ResNet
    train_data = train.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid1', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_1'})
    train_data = train_data.drop(columns=['variantid'])

    train_data = train_data.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid2', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_2'})
    train_data = train_data.drop(columns=['variantid'])

    # Объединение с текстовыми данными и эмбеддингами BERT
    train_data = train_data.merge(text_and_bert[['variantid', 'combined_text', 'name_bert_64']], left_on='variantid1', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'combined_text': 'text_1', 'name_bert_64': 'text_embedding_1'})
    train_data = train_data.drop(columns=['variantid'])

    train_data = train_data.merge(text_and_bert[['variantid', 'combined_text', 'name_bert_64']], left_on='variantid2', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'combined_text': 'text_2', 'name_bert_64': 'text_embedding_2'})
    train_data = train_data.drop(columns=['variantid'])

    # Удаление строк, в которых есть пропущенные значения
    train_data = train_data.dropna()

    return train_data

# Объединение эмбеддингов
def combine_embeddings(row):
    # Объединение эмбеддингов изображений и текста
    pic_embeddings = np.concatenate([row['pic_embeddings_1'][0], row['pic_embeddings_2'][0]])
    text_embeddings = np.concatenate([row['text_embedding_1'], row['text_embedding_2']])
    return np.concatenate([pic_embeddings, text_embeddings])

# Подготовка данных для обучения модели
def prepare_data(train_data):
    # Объединение эмбеддингов изображений и текстов
    train_data['combined_embeddings'] = train_data.apply(combine_embeddings, axis=1)

    # Преобразование объединенных эмбеддингов в массив numpy
    X = np.vstack(train_data['combined_embeddings'].values)
    y = train_data['target']

    # Разделение данных на обучающую и валидационную выборки
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=23)

    return X_train, X_val, y_train, y_val

# Обучение модели
def train_model(X_train, y_train):
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    # Сохранение обученной модели
    joblib.dump(model, 'random_forest.pkl')
    return model

# Оценка модели
def evaluate_model(model, X_val, y_val):
    y_pred_prob = model.predict_proba(X_val)[:, 1]
    y_pred = (y_pred_prob >= 0.5).astype(int)

    precision, recall, _ = precision_recall_curve(y_val, y_pred_prob)
    prauc = auc(recall, precision)
    print(f'PRAUC: {prauc}')

    # Сохранение векторизатора, если таковой используется
    # Пример:
    # joblib.dump(tfidf_vectorizer, 'vectorizer1.pk3')

# Главная функция для запуска всего процесса
def main():
    attributes, resnet, text_and_bert, train, test = load_data()

    # Обработка текстовых данных
    text_and_bert = process_text_and_bert(text_and_bert)

    # Объединение данных
    train_data = merge_data(train, resnet, text_and_bert)

    # Подготовка данных для обучения модели
    X_train, X_val, y_train, y_val = prepare_data(train_data)

    # Обучение модели
    model = train_model(X_train, y_train)
    
    # Сохранение модели
    joblib.dump(model, 'random_forest1.pk3')

    # Оценка модели
    evaluate_model(model, X_val, y_val)

    # Закомментированный код для CatBoost
    # from catboost import CatBoostClassifier
    # model_catboost = CatBoostClassifier(iterations=1000, depth=10, learning_rate=0.1, loss_function='Logloss', eval_metric='AUC')
    # model_catboost.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=True)
    # joblib.dump(model_catboost, 'catboost_model.pkl')

if __name__ == "__main__":
    main()


PRAUC: 0.5936540981777991


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import precision_recall_curve, auc
import joblib
import json

# Путь к данным  
attributes_path = 'F:/competition/attributes.parquet'  
resnet_path = 'F:/competition/resnet.parquet'  
text_and_bert_path = 'F:/competition/text_and_bert.parquet'  
train_path = 'F:/competition/train.parquet'  
test_path = 'F:/competition/test.parquet'  

# Функция для загрузки данных  
def load_data(sample_size=0.1):  
    # Загрузка данных из файлов Parquet  
    attributes = pd.read_parquet(attributes_path)  
    resnet = pd.read_parquet(resnet_path)  
    text_and_bert = pd.read_parquet(text_and_bert_path)  
    train = pd.read_parquet(train_path)  
    test = pd.read_parquet(test_path)  

    # Если sample_size меньше 1, то выборка данных  
    if sample_size < 1:  
        attributes = attributes.sample(frac=sample_size, random_state=42)  
        resnet = resnet.sample(frac=sample_size, random_state=42)  
        text_and_bert = text_and_bert.sample(frac=sample_size, random_state=42)  
        train = train.sample(frac=sample_size, random_state=42)  
        test = test.sample(frac=sample_size, random_state=42)  

    # Возвращаем 5 DataFrame'ов
    return attributes, resnet, text_and_bert, train, test  

# Обработка текстовых данных
def process_text_and_bert(df):
    # Объединение столбцов name и description в единый текст
    df['combined_text'] = df['name'] + ' ' + df['description']
    return df

# Объединение данных
def merge_data(train, resnet, text_and_bert):
    # Объединение с эмбеддингами ResNet
    train_data = train.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid1', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_1'})
    train_data = train_data.drop(columns=['variantid'])

    train_data = train_data.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid2', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_2'})
    train_data = train_data.drop(columns=['variantid'])

    # Объединение с текстовыми данными и эмбеддингами BERT
    train_data = train_data.merge(text_and_bert[['variantid', 'combined_text', 'name_bert_64']], left_on='variantid1', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'combined_text': 'text_1', 'name_bert_64': 'text_embedding_1'})
    train_data = train_data.drop(columns=['variantid'])

    train_data = train_data.merge(text_and_bert[['variantid', 'combined_text', 'name_bert_64']], left_on='variantid2', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'combined_text': 'text_2', 'name_bert_64': 'text_embedding_2'})
    train_data = train_data.drop(columns=['variantid'])

    # Удаление строк, в которых есть пропущенные значения
    train_data = train_data.dropna()

    return train_data

# Объединение эмбеддингов
def combine_embeddings(row):
    # Объединение эмбеддингов изображений и текста
    pic_embeddings = np.concatenate([row['pic_embeddings_1'][0], row['pic_embeddings_2'][0]])
    text_embeddings = np.concatenate([row['text_embedding_1'], row['text_embedding_2']])
    return np.concatenate([pic_embeddings, text_embeddings])

# Подготовка данных для обучения модели
def prepare_data(train_data):
    # Объединение эмбеддингов изображений и текстов
    train_data['combined_embeddings'] = train_data.apply(combine_embeddings, axis=1)

    # Преобразование объединенных эмбеддингов в массив numpy
    X = np.vstack(train_data['combined_embeddings'].values)
    y = train_data['target']

    # Разделение данных на обучающую и валидационную выборки
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=23)

    return X_train, X_val, y_train, y_val

# Обучение модели
def train_model(X_train, y_train):
    model = CatBoostClassifier(iterations=1000, depth=10, learning_rate=0.1, loss_function='Logloss', eval_metric='AUC', verbose=True)
    model.fit(X_train, y_train)
    
    # Сохранение обученной модели
    joblib.dump(model, 'catboost_model1.pkl')
    return model

# Оценка модели
def evaluate_model(model, X_val, y_val):
    y_pred_prob = model.predict_proba(X_val)[:, 1]
    y_pred = (y_pred_prob >= 0.5).astype(int)

    precision, recall, _ = precision_recall_curve(y_val, y_pred_prob)
    prauc = auc(recall, precision)
    print(f'PRAUC: {prauc}')

    # Сохранение модели
    joblib.dump(model, 'catboost_model1.pkl')

    # Сохранение векторизатора (если используется), в данном случае он не используется
    # joblib.dump(tfidf_vectorizer, 'vectorizer1.pk3')

# Главная функция для запуска всего процесса
def main():
    attributes, resnet, text_and_bert, train, test = load_data()

    # Обработка текстовых данных
    text_and_bert = process_text_and_bert(text_and_bert)

    # Объединение данных
    train_data = merge_data(train, resnet, text_and_bert)

    # Подготовка данных для обучения модели
    X_train, X_val, y_train, y_val = prepare_data(train_data)

    # Обучение модели
    model = train_model(X_train, y_train)
    
    # Оценка модели
    evaluate_model(model, X_val, y_val)

if __name__ == "__main__":
    main()


0:	total: 2s	remaining: 33m 23s
1:	total: 3.86s	remaining: 32m 4s
2:	total: 5.62s	remaining: 31m 7s
3:	total: 7.41s	remaining: 30m 45s
4:	total: 9.19s	remaining: 30m 28s
5:	total: 11s	remaining: 30m 20s
6:	total: 12.9s	remaining: 30m 27s
7:	total: 14.7s	remaining: 30m 24s
8:	total: 16.6s	remaining: 30m 26s
9:	total: 18.4s	remaining: 30m 21s
10:	total: 20.1s	remaining: 30m 11s
11:	total: 22s	remaining: 30m 13s
12:	total: 23.9s	remaining: 30m 18s
13:	total: 25.8s	remaining: 30m 14s
14:	total: 27.5s	remaining: 30m 8s
15:	total: 29.3s	remaining: 30m
16:	total: 31.1s	remaining: 29m 58s
17:	total: 33s	remaining: 29m 58s
18:	total: 34.7s	remaining: 29m 51s
19:	total: 36.5s	remaining: 29m 47s
20:	total: 38.3s	remaining: 29m 44s
21:	total: 40.1s	remaining: 29m 41s
22:	total: 41.8s	remaining: 29m 36s
23:	total: 43.6s	remaining: 29m 33s
24:	total: 45.4s	remaining: 29m 31s
25:	total: 47.3s	remaining: 29m 31s
26:	total: 49s	remaining: 29m 27s
27:	total: 50.8s	remaining: 29m 22s
28:	total: 52.6s	rem

In [ ]:
import pandas as pd  
import numpy as np  
from sklearn.model_selection import train_test_split  
from sklearn.linear_model import LogisticRegression  
from sklearn.metrics import precision_recall_curve, auc  
from sklearn.feature_extraction.text import TfidfVectorizer  
import joblib  
import json  

# # Путь к данным  
# attributes_path = 'F:/competition/attributes.parquet'  
# resnet_path = 'F:/competition/resnet.parquet'  
# text_and_bert_path = 'F:/competition/text_and_bert.parquet'  
# train_path = 'F:/competition/train.parquet'  
# test_path = 'F:/competition/test.parquet'  

# # Функция для загрузки данных  
# def load_data(sample_size=0.1):  
#     # Загрузка данных из файлов Parquet  
#     attributes = pd.read_parquet(attributes_path)  
#     resnet = pd.read_parquet(resnet_path)  
#     text_and_bert = pd.read_parquet(text_and_bert_path)  
#     train = pd.read_parquet(train_path)  
#     test = pd.read_parquet(test_path)  

#     # Если sample_size меньше 1, то выборка данных  
#     if sample_size < 1:  
#         attributes = attributes.sample(frac=sample_size, random_state=42)  
#         resnet = resnet.sample(frac=sample_size, random_state=42)  
#         text_and_bert = text_and_bert.sample(frac=sample_size, random_state=42)  
#         train = train.sample(frac=sample_size, random_state=42)  
#         test = test.sample(frac=sample_size, random_state=42)  

#     return attributes, resnet, text_and_bert, train, test  

# # Загрузка данных  
# attributes, resnet, text_and_bert, train, test = load_data()  

# Проверка наличия необходимых столбцов перед объединением  
def load_parquet_file(path, expected_columns):  
    """Функция для загрузки данных из файла Parquet с проверкой столбцов."""  
    try:  
        df = pd.read_parquet(path)  
        missing_cols = [col for col in expected_columns if col not in df.columns]  
        if missing_cols:  
            raise ValueError(f"Отсутствуют столбцы в файле {path}: {', '.join(missing_cols)}")  
        return df  
    except Exception as e:  
        print(f"Ошибка при загрузке {path}: {e}")  
        return pd.DataFrame()  # Возвращаем пустой DataFrame в случае ошибки  

# Загрузка данных с проверкой отсутствующих колонок  
attributes = load_parquet_file(attributes_path, ['variantid', 'categories', 'characteristic_attributes_mapping'])  
resnet = load_parquet_file(resnet_path, ['variantid', 'main_pic_embeddings_resnet_v1'])  
text_and_bert = load_parquet_file(text_and_bert_path, ['variantid', 'combined_text'])  
train = load_parquet_file(train_path, ['variantid1', 'variantid2', 'target'])  

# Обработка атрибутов  
if not attributes.empty:  
    # Проверка наличия столбца 'categories'  
    if 'categories' in attributes.columns:  
        attributes['categories'] = attributes['categories'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)  
    else:  
        print("Столбец 'categories' отсутствует в данных атрибутов.")  
    
    if 'characteristic_attributes_mapping' in attributes.columns:  
        attributes['characteristic_attributes_mapping'] = attributes['characteristic_attributes_mapping'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)  
    else:  
        print("Столбец 'characteristic_attributes_mapping' отсутствует в данных атрибутов.")  
    
    def extract_text_from_row(row):  
        category_text = ' '.join(  
            [' '.join(map(str, v)) if isinstance(v, list) else str(v) for v in list(row['categories'].values())]  
        )  
        attributes_text = ' '.join(  
            [' '.join(map(str, v)) if isinstance(v, list) else str(v) for v in list(row['characteristic_attributes_mapping'].values())]  
        )  
        return f"{category_text} {attributes_text}"  



        # Создание комбинированного текста  
    attributes['combined_text'] = attributes.apply(extract_text_from_row, axis=1)  

# Объединение данных для обучения  
if not train.empty and not resnet.empty and not attributes.empty:  
    train_data = train.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid1', right_on='variantid', how='left')  
    train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_1'})  
    train_data = train_data.drop(columns=['variantid'])  

    train_data = train_data.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid2', right_on='variantid', how='left')  
    train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_2'})  
    train_data = train_data.drop(columns=['variantid'])  

    train_data = train_data.merge(attributes[['variantid', 'combined_text']], left_on='variantid1', right_on='variantid', how='left')  
    train_data = train_data.rename(columns={'combined_text': 'text_1'})  
    train_data = train_data.drop(columns=['variantid'])  

    train_data = train_data.merge(attributes[['variantid', 'combined_text']], left_on='variantid2', right_on='variantid', how='left')  
    train_data = train_data.rename(columns={'combined_text': 'text_2'})  
    train_data = train_data.drop(columns=['variantid'])  

    train_data = train_data.dropna()  # Удаление строк с пропущенными значениями  
else:  
    train_data = pd.DataFrame()  

# Шаг 4: Подготовка данных  
if not train_data.empty:  
    # Создание объекта TfidfVectorizer  
    tfidf_vectorizer = TfidfVectorizer(max_features=2000)  

    # Преобразование текстовых данных в эмбеддинги  
    text_data = train_data['text_1'].fillna('') + ' ' + train_data['text_2'].fillna('')  
    text_embeddings = tfidf_vectorizer.fit_transform(text_data).toarray()  

    # Функция для объединения эмбеддингов изображений и текста  
    def combine_embeddings(row):  
        pic_embeddings = np.concatenate([row['pic_embeddings_1'][0], row['pic_embeddings_2'][0]])  
        text_embeddings_row = text_embeddings[row.name]  
        return np.concatenate([pic_embeddings, text_embeddings_row])  

    # Применение функции к данным  
    train_data['combined_embeddings'] = train_data.apply(combine_embeddings, axis=1)  

    # Создание матрицы признаков и вектора целей  
    X = np.vstack(train_data['combined_embeddings'].values)  
    y = train_data['target']  

    # Разделение данных на обучающую и валидационную выборки  
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.12, random_state=42)  
else:  
    X_train, X_val, y_train, y_val = (np.array([]),) * 4  

# Шаг 5: Обучение модели  
if X_train.size > 0 and y_train.size > 0:  
    # Создание и обучение модели логистической регрессии  
    model = LogisticRegression(max_iter=2000)  
    model.fit(X_train, y_train)  

    # Сохранение модели  
    joblib.dump(model, 'baseline1.pk3')  

# Шаг 6: Оценка модели  
if X_val.size > 0 and y_val.size > 0:  
    # Прогнозирование вероятностей  
    y_pred_prob = model.predict_proba(X_val)[:, 1]  
    y_pred = (y_pred_prob >= 0.5).astype(int)  

    # Вычисление метрик точности и полноты  
    precision, recall, _ = precision_recall_curve(y_val, y_pred_prob)  
    prauc = auc(recall, precision)  
    print(f'PRAUC: {prauc}')  

    # Сохранение вектораизатора  
    joblib.dump(tfidf_vectorizer, 'vectorizer1.pk3')

Ошибка при загрузке F:/competition/resnet.parquet: malloc of size 17179869184 failed
Ошибка при загрузке F:/competition/text_and_bert.parquet: Отсутствуют столбцы в файле F:/competition/text_and_bert.parquet: combined_text


PRAUC: 0.7724866290617343

In [1]:
import pandas as pd  
import numpy as np  
from sklearn.model_selection import train_test_split  
from sklearn.linear_model import LogisticRegression  
from sklearn.metrics import precision_recall_curve, auc  
from sklearn.feature_extraction.text import TfidfVectorizer  
import joblib  
import json  

# Путь к данным  
attributes_path = 'F:/competition/attributes.parquet'  
resnet_path = 'F:/competition/resnet.parquet'  
text_and_bert_path = 'F:/competition/text_and_bert.parquet'  
train_path = 'F:/competition/train.parquet'  
test_path = 'F:/competition/test.parquet'  

# Функция для загрузки данных  
def load_data(sample_size=0.1):  
    # Загрузка данных из файлов Parquet  
    attributes = pd.read_parquet(attributes_path)  
    resnet = pd.read_parquet(resnet_path)  
    text_and_bert = pd.read_parquet(text_and_bert_path)  
    train = pd.read_parquet(train_path)  
    test = pd.read_parquet(test_path)  

    # Если sample_size меньше 1, то выборка данных  
    if sample_size < 1:  
        attributes = attributes.sample(frac=sample_size, random_state=42)  
        resnet = resnet.sample(frac=sample_size, random_state=42)  
        text_and_bert = text_and_bert.sample(frac=sample_size, random_state=42)  
        train = train.sample(frac=sample_size, random_state=42)  
        test = test.sample(frac=sample_size, random_state=42)  

    return attributes, resnet, text_and_bert, train, test  

# Загрузка данных  
attributes, resnet, text_and_bert, train, test = load_data()  

# Проверка наличия необходимых столбцов перед объединением  
def load_parquet_file(path, expected_columns):  
    """Функция для загрузки данных из файла Parquet с проверкой столбцов."""  
    try:  
        df = pd.read_parquet(path)  
        missing_cols = [col for col in expected_columns if col not in df.columns]  
        if missing_cols:  
            raise ValueError(f"Отсутствуют столбцы в файле {path}: {', '.join(missing_cols)}")  
        return df  
    except Exception as e:  
        print(f"Ошибка при загрузке {path}: {e}")  
        return pd.DataFrame()  # Возвращаем пустой DataFrame в случае ошибки  

# Загрузка данных с проверкой отсутствующих колонок  
attributes = load_parquet_file('F:/competition/attributes.parquet', ['variantid', 'categories', 'characteristic_attributes_mapping'])  
resnet = load_parquet_file('F:/competition/resnet.parquet', ['variantid', 'main_pic_embeddings_resnet_v1'])  
text_and_bert = load_parquet_file('F:/competition/text_and_bert.parquet', ['variantid', 'combined_text'])  
train = load_parquet_file('F:/competition/train.parquet', ['variantid1', 'variantid2', 'target'])  

# Обработка атрибутов  
if not attributes.empty:  
    # Проверка наличия столбца 'categories'  
    if 'categories' in attributes.columns:  
        attributes['categories'] = attributes['categories'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)  
    else:  
        print("Столбец 'categories' отсутствует в данных атрибутов.")  
    
    if 'characteristic_attributes_mapping' in attributes.columns:  
        attributes['characteristic_attributes_mapping'] = attributes['characteristic_attributes_mapping'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)  
    else:  
        print("Столбец 'characteristic_attributes_mapping' отсутствует в данных атрибутов.")  
    
    def extract_text_from_row(row):  
        category_text = ' '.join(  
            [' '.join(map(str, v)) if isinstance(v, list) else str(v) for v in list(row['categories'].values())]  
        )  
        attributes_text = ' '.join(  
            [' '.join(map(str, v)) if isinstance(v, list) else str(v) for v in list(row['characteristic_attributes_mapping'].values())]  
        )  
        return f"{category_text} {attributes_text}"  

    # Создание комбинированного текста  
    attributes['combined_text'] = attributes.apply(extract_text_from_row, axis=1)  

# Объединение данных для обучения  
if not train.empty and not resnet.empty and not attributes.empty:  
    try:
        train_data = train.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid1', right_on='variantid', how='left')  
        train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_1'})  
        train_data = train_data.drop(columns=['variantid'])  

        train_data = train_data.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid2', right_on='variantid', how='left')  
        train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_2'})  
        train_data = train_data.drop(columns=['variantid'])  

        train_data = train_data.merge(attributes[['variantid', 'combined_text']], left_on='variantid1', right_on='variantid', how='left')  
        train_data = train_data.rename(columns={'combined_text': 'text_1'})  
        train_data = train_data.drop(columns=['variantid'])  

        train_data = train_data.merge(attributes[['variantid', 'combined_text']], left_on='variantid2', right_on='variantid', how='left')  
        train_data = train_data.rename(columns={'combined_text': 'text_2'})  
        train_data = train_data.drop(columns=['variantid'])  

        train_data = train_data.dropna()  # Удаление строк с пропущенными значениями  
    except Exception as e:
        print(f"Ошибка при объединении данных: {e}")
        train_data = pd.DataFrame()  

# Шаг 4: Подготовка данных  
if not train_data.empty:  
    # Создание объекта TfidfVectorizer  
    tfidf_vectorizer = TfidfVectorizer(max_features=2000)  

    # Преобразование текстовых данных в эмбеддинги  
    text_data = train_data['text_1'].fillna('') + ' ' + train_data['text_2'].fillna('')  
    text_embeddings = tfidf_vectorizer.fit_transform(text_data).toarray()  

    # Функция для объединения эмбеддингов изображений и текста  
    def combine_embeddings(row):  
        pic_embeddings = np.concatenate([row['pic_embeddings_1'][0], row['pic_embeddings_2'][0]])  
        text_embeddings_row = text_embeddings[row.name]  
        return np.concatenate([pic_embeddings, text_embeddings_row])  

    # Применение функции к данным  
    train_data['combined_embeddings'] = train_data.apply(combine_embeddings, axis=1)  

    # Создание матрицы признаков и вектора целей  
    X = np.vstack(train_data['combined_embeddings'].values)  
    y = train_data['target']  

    # Разделение данных на обучающую и валидационную выборки  
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.12, random_state=42)  
else:  
    X_train, X_val, y_train, y_val = (np.array([]),) * 4  

# Шаг 5: Обучение модели  
if X_train.size > 0 and y_train.size > 0:  
    # Создание и обучение модели логистической регрессии  
    model = LogisticRegression(max_iter=2000)  
    model.fit(X_train, y_train)  

    # Сохранение модели  
    joblib.dump(model, 'baseline1.pk3')  

# Шаг 6: Оценка модели  
if X_val.size > 0 and y_val.size > 0:  
    # Прогнозирование вероятностей  
    y_pred_prob = model.predict_proba(X_val)[:, 1]  
    y_pred = (y_pred_prob >= 0.5).astype(int)  

    # Вычисление метрик точности и полноты  
    precision, recall, _ = precision_recall_curve(y_val, y_pred_prob)  
    prauc = auc(recall, precision)  
    print(f'PRAUC: {prauc}')  

    # Сохранение вектораизатора  
    joblib.dump(tfidf_vectorizer, 'vectorizer1.pk3')  


Ошибка при загрузке F:/competition/text_and_bert.parquet: Отсутствуют столбцы в файле F:/competition/text_and_bert.parquet: combined_text
PRAUC: 0.7713306247507894


In [5]:
import pandas as pd  
import numpy as np  
from sklearn.model_selection import train_test_split  
from sklearn.linear_model import LogisticRegression  
from sklearn.metrics import precision_recall_curve, auc  
from sklearn.feature_extraction.text import TfidfVectorizer  
import joblib  
import json  

# Путь к данным  
attributes_path = 'F:/competition/attributes.parquet'  
resnet_path = 'F:/competition/resnet.parquet'  
text_and_bert_path = 'F:/competition/text_and_bert.parquet'  
train_path = 'F:/competition/train.parquet'  
test_path = 'F:/competition/test.parquet'  

# Функция для загрузки данных  
def load_data(sample_size=0.1):  
    # Загрузка данных из файлов Parquet  
    attributes = pd.read_parquet(attributes_path)  
    resnet = pd.read_parquet(resnet_path)  
    text_and_bert = pd.read_parquet(text_and_bert_path)  
    train = pd.read_parquet(train_path)  
    test = pd.read_parquet(test_path)  

    # Если sample_size меньше 1, то выборка данных  
    if sample_size < 1:  
        attributes = attributes.sample(frac=sample_size, random_state=42)  
        resnet = resnet.sample(frac=sample_size, random_state=42)  
        text_and_bert = text_and_bert.sample(frac=sample_size, random_state=42)  
        train = train.sample(frac=sample_size, random_state=42)  
        test = test.sample(frac=sample_size, random_state=42)  

    return attributes, resnet, text_and_bert, train, test  

# Загрузка данных  
attributes, resnet, text_and_bert, train, test = load_data()  


# Загрузка данных с проверкой отсутствующих колонок  
def load_parquet_file(path, expected_columns):  
    """Функция для загрузки данных из файла Parquet с проверкой столбцов."""  
    try:  
        df = pd.read_parquet(path)  
        missing_cols = [col for col in expected_columns if col not in df.columns]  
        if missing_cols:  
            print(f"Отсутствуют столбцы в файле {path}: {', '.join(missing_cols)}")  
            # Создаем отсутствующие столбцы с пустыми значениями  
            for col in missing_cols:  
                df[col] = np.nan  
        return df  
    except Exception as e:  
        print(f"Ошибка при загрузке {path}: {e}")  
        return pd.DataFrame()  # Возвращаем пустой DataFrame в случае ошибки  

# Загрузка данных с проверкой отсутствующих колонок  
attributes = load_parquet_file(attributes_path, ['variantid', 'categories', 'characteristic_attributes_mapping'])  
resnet = load_parquet_file(resnet_path, ['variantid', 'main_pic_embeddings_resnet_v1'])  
text_and_bert = load_parquet_file(text_and_bert_path, ['variantid', 'combined_text'])  
train = load_parquet_file(train_path, ['variantid1', 'variantid2', 'target'])  

# Обработка атрибутов  
if not attributes.empty:  
    # Проверка наличия столбца 'categories'  
    if 'categories' in attributes.columns:  
        attributes['categories'] = attributes['categories'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)  
    else:  
        print("Столбец 'categories' отсутствует в данных атрибутов.")  
    
    if 'characteristic_attributes_mapping' in attributes.columns:  
        attributes['characteristic_attributes_mapping'] = attributes['characteristic_attributes_mapping'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)  
    else:  
        print("Столбец 'characteristic_attributes_mapping' отсутствует в данных атрибутов.")  
    
    def extract_text_from_row(row):  
        category_text = ' '.join(  
            [' '.join(map(str, v)) if isinstance(v, list) else str(v) for v in list(row['categories'].values())]  
        )  
        attributes_text = ' '.join(  
            [' '.join(map(str, v)) if isinstance(v, list) else str(v) for v in list(row['characteristic_attributes_mapping'].values())]  
        )  
        return f"{category_text} {attributes_text}"  

    # Создание комбинированного текста  
    attributes['combined_text'] = attributes.apply(extract_text_from_row, axis=1)  

# Объединение данных для обучения  
if not train.empty and not resnet.empty and not attributes.empty:  
    train_data = train.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid1', right_on='variantid', how='left')  
    train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_1'})  
    train_data = train_data.drop(columns=['variantid'])  

    train_data = train_data.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid2', right_on='variantid', how='left')  
    train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_2'})  
    train_data = train_data.drop(columns=['variantid'])  

    train_data = train_data.merge(attributes[['variantid', 'combined_text']], left_on='variantid1', right_on='variantid', how='left')  
    train_data = train_data.rename(columns={'combined_text': 'text_1'})  
    train_data = train_data.drop(columns=['variantid'])  

    train_data = train_data.merge(attributes[['variantid', 'combined_text']], left_on='variantid2', right_on='variantid', how='left')  
    train_data = train_data.rename(columns={'combined_text': 'text_2'})  
    train_data = train_data.drop(columns=['variantid'])  

    train_data = train_data.dropna()  # Удаление строк с пропущенными значениями  
else:  
    train_data = pd.DataFrame() 
  

# Шаг 4: Подготовка данных  
if not train_data.empty:  
    # Создание объекта TfidfVectorizer  
    tfidf_vectorizer = TfidfVectorizer(max_features=2000)  

    # Преобразование текстовых данных в эмбеддинги  
    text_data = train_data['text_1'].fillna('') + ' ' + train_data['text_2'].fillna('')  
    text_embeddings = tfidf_vectorizer.fit_transform(text_data).toarray()  

    # Функция для объединения эмбеддингов изображений и текста  
    def combine_embeddings(row):  
        pic_embeddings = np.concatenate([row['pic_embeddings_1'][0], row['pic_embeddings_2'][0]])  
        text_embeddings_row = text_embeddings[row.name]  
        return np.concatenate([pic_embeddings, text_embeddings_row])  

    # Применение функции к данным  
    train_data['combined_embeddings'] = train_data.apply(combine_embeddings, axis=1)  

    # Создание матрицы признаков и вектора целей  
    X = np.vstack(train_data['combined_embeddings'].values)  
    y = train_data['target']  

    # Разделение данных на обучающую и валидационную выборки  
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=23)  
else:  
    X_train, X_val, y_train, y_val = (np.array([]),) * 4  

                                  

# Шаг 5: Обучение модели  
if X_train.size > 0 and y_train.size > 0:  
    # Создание и обучение модели логистической регрессии  
    model = LogisticRegression(max_iter=2000)  
    model.fit(X_train, y_train)  

    # Сохранение модели  
    joblib.dump(model, 'baseline0.pkl')  

# Шаг 6: Оценка модели  
if X_val.size > 0 and y_val.size > 0:  
    # Прогнозирование вероятностей  
    y_pred_prob = model.predict_proba(X_val)[:, 1]  
    y_pred = (y_pred_prob >= 0.5).astype(int)  

    # Вычисление метрик точности и полноты  
    precision, recall, _ = precision_recall_curve(y_val, y_pred_prob)  
    prauc = auc(recall, precision)  
    print(f'PRAUC: {prauc}')  

    # Сохранение вектораизатора  
    joblib.dump(tfidf_vectorizer, 'vectorizer0.pkl')

Отсутствуют столбцы в файле F:/competition/text_and_bert.parquet: combined_text
PRAUC: 0.7724866290617343


In [8]:
import pandas as pd  
import numpy as np  
from sklearn.model_selection import train_test_split  
from sklearn.linear_model import LogisticRegression  
from sklearn.metrics import precision_recall_curve, auc  
from sklearn.feature_extraction.text import TfidfVectorizer  
import joblib  
import json  

# Путь к данным  
attributes_path = 'F:/competition/attributes.parquet'  
resnet_path = 'F:/competition/resnet.parquet'  
text_and_bert_path = 'F:/competition/text_and_bert.parquet'  
train_path = 'F:/competition/train.parquet'  
test_path = 'F:/competition/test.parquet'  

# Функция для загрузки данных  
def load_data(sample_size=0.1):  
    # Загрузка данных из файлов Parquet  
    attributes = pd.read_parquet(attributes_path)  
    resnet = pd.read_parquet(resnet_path)  
    text_and_bert = pd.read_parquet(text_and_bert_path)  
    train = pd.read_parquet(train_path)  
    test = pd.read_parquet(test_path)  

    # Если sample_size меньше 1, то выборка данных  
    if sample_size < 1:  
        attributes = attributes.sample(frac=sample_size, random_state=42)  
        resnet = resnet.sample(frac=sample_size, random_state=42)  
        text_and_bert = text_and_bert.sample(frac=sample_size, random_state=42)  
        train = train.sample(frac=sample_size, random_state=42)  
        test = test.sample(frac=sample_size, random_state=42)  

    return attributes, resnet, text_and_bert, train, test  

# Загрузка данных  
attributes, resnet, text_and_bert, train, test = load_data()  

# Проверка наличия необходимых столбцов перед объединением  
def load_parquet_file(path, expected_columns):  
    """Функция для загрузки данных из файла Parquet с проверкой столбцов."""  
    try:  
        df = pd.read_parquet(path)  
        missing_cols = [col for col in expected_columns if col not in df.columns]  
        if missing_cols:  
            raise ValueError(f"Отсутствуют столбцы в файле {path}: {', '.join(missing_cols)}")  
        return df  
    except Exception as e:  
        print(f"Ошибка при загрузке {path}: {e}")  
        return pd.DataFrame()  # Возвращаем пустой DataFrame в случае ошибки  

# Загрузка данных с проверкой отсутствующих колонок  
attributes = load_parquet_file(attributes_path, ['variantid', 'categories', 'characteristic_attributes_mapping'])  
resnet = load_parquet_file(resnet_path, ['variantid', 'main_pic_embeddings_resnet_v1'])  
text_and_bert = load_parquet_file(text_and_bert_path, ['variantid', 'combined_text'])  
train = load_parquet_file(train_path, ['variantid1', 'variantid2', 'target'])  

# Обработка атрибутов  
if not attributes.empty:  
    attributes['categories'] = attributes['categories'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)  
    attributes['characteristic_attributes_mapping'] = attributes['characteristic_attributes_mapping'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)  

    def extract_text_from_row(row):  
        category_text = ' '.join(  
            [' '.join(map(str, v)) if isinstance(v, list) else str(v) for v in list(row['categories'].values())]  
        )  
        attributes_text = ' '.join(  
            [' '.join(map(str, v)) if isinstance(v, list) else str(v) for v in list(row['characteristic_attributes_mapping'].values())]  
        )  
        return f"{category_text} {attributes_text}"  

    # Создание комбинированного текста  
    attributes['combined_text'] = attributes.apply(extract_text_from_row, axis=1)  

# Объединение данных для обучения  
if not train.empty and not resnet.empty and not attributes.empty:  
    train_data = train.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid1', right_on='variantid', how='left')  
    train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_1'})  
    train_data = train_data.drop(columns=['variantid'])  

    train_data = train_data.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid2', right_on='variantid', how='left')  
    train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_2'})  
    train_data = train_data.drop(columns=['variantid'])  

    train_data = train_data.merge(attributes[['variantid', 'combined_text']], left_on='variantid1', right_on='variantid', how='left')  
    train_data = train_data.rename(columns={'combined_text': 'text_1'})  
    train_data = train_data.drop(columns=['variantid'])  

    train_data = train_data.merge(attributes[['variantid', 'combined_text']], left_on='variantid2', right_on='variantid', how='left')  
    train_data = train_data.rename(columns={'combined_text': 'text_2'})  
    train_data = train_data.drop(columns=['variantid'])  

    train_data = train_data.dropna()  # Удаление строк с пропущенными значениями  
else:  
    train_data = pd.DataFrame()  

# Шаг 4: Подготовка данных  
if not train_data.empty:  
    # Создание объекта TfidfVectorizer  
    tfidf_vectorizer = TfidfVectorizer(max_features=2000)  

    # Преобразование текстовых данных в эмбеддинги  
    text_data = train_data['text_1'].fillna('') + ' ' + train_data['text_2'].fillna('')  
    text_embeddings = tfidf_vectorizer.fit_transform(text_data).toarray()  

    # Функция для объединения эмбеддингов изображений и текста  
    def combine_embeddings(row):  
        pic_embeddings = np.concatenate([row['pic_embeddings_1'][0], row['pic_embeddings_2'][0]])  
        text_embeddings_row = text_embeddings[row.name]  
        return np.concatenate([pic_embeddings, text_embeddings_row])  

    # Применение функции к данным  
    train_data['combined_embeddings'] = train_data.apply(combine_embeddings, axis=1)  

    # Создание матрицы признаков и вектора целей  
    X = np.vstack(train_data['combined_embeddings'].values)  
    y = train_data['target']  

    # Разделение данных на обучающую и валидационную выборки  
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=23)  
else:  
    X_train, X_val, y_train, y_val = (np.array([]),) * 4  

# Шаг 5: Обучение модели  
if X_train.size > 0 and y_train.size > 0:  
    # Создание и обучение модели логистической регрессии  
    model = LogisticRegression(max_iter=2000)  
    model.fit(X_train, y_train)  

    # Сохранение модели  
    joblib.dump(model, 'baseline1.pkl')  

# Шаг 6: Оценка модели  
if X_val.size > 0 and y_val.size > 0:  
    # Прогнозирование вероятностей  
    y_pred_prob = model.predict_proba(X_val)[:, 1]  
    y_pred = (y_pred_prob >= 0.5).astype(int)  

    # Вычисление метрик точности и полноты  
    precision, recall, _ = precision_recall_curve(y_val, y_pred_prob)  
    prauc = auc(recall, precision)  
    print(f'PRAUC: {prauc}')  

    # Сохранение вектораизатора  
    joblib.dump(tfidf_vectorizer, 'vectorizer1.pkl')

Ошибка при загрузке F:/competition/text_and_bert.parquet: Отсутствуют столбцы в файле F:/competition/text_and_bert.parquet: combined_text
PRAUC: 0.7724866290617343



Text and Bert DataFrame:


,variantid,name,description,name_bert_64
0,47920382,"Игрушка для ванной , сувенир Уточка Дьяволица",Серия уточек бренда FUNNY DUCKS представлена ...,"[-0.24964939057826996, 0.6433379650115967, 0.4..."
1,49801845,Стразы(бусины) клеевые на листе 9*16 см,Стразы(бусины) клеевые на листе 9*16 см,"[-0.6397916078567505, 0.3660058379173279, 0.67..."
2,49853444,"Набор для вышивания Vervaco ""Подушка. Геометри...",Состав набора: канва-страмин Zweigart с нанесе...,"[-0.27170804142951965, 0.3709857165813446, 0.4..."
3,49893028,"Кружево коклюшечное ""Prym"", цвет: серый, 8 мм,...","Кружево коклюшечное ""Prym"" предназначено для о...","[-0.5911799669265747, 0.45404374599456787, 0.4..."
4,49987483,Оригами Настольная игра Фиксики Кодовый замок,"В настольной игре Оригами ""Фиксики. Кодовый за...","[-0.29846125841140747, 0.41105785965919495, 0...."
...,...,...,...,...
2252564,1799186009,"ЭлектросамокатSKU-000226hb, черный",Введение в продукт<br/><br/>Название продукта:...,"[-0.5560247898101807, 0.4461979568004608, 0.47..."
2252565,1801305376,Золотой человек | Дик Филип Киндред,«…Выворачивать наизнанку базовые категории реа...,"[-0.4282682240009308, 0.49429962038993835, 0.5..."
2252566,1802944592,"Daiwa Монофильная леска для рыбалки, размотка:...",None,"[-0.5640578866004944, 0.13208043575286865, 0.6..."
2252567,1803038155,Шарлатанка в Академии драконов | Миш Виктория,"Тяжело поступать адекватно, когда тебя «любящи...","[-0.35129043459892273, 0.48004797101020813, 0...."



Train DataFrame:


,variantid1,variantid2,target
0,1447875869,1447872068,1
1,1176231201,284733670,1
2,658617865,549848659,0
3,719320625,719370486,1
4,1067645658,949954740,0
...,...,...,...
1168511,970087885,944020046,0
1168512,834554806,118393780,1
1168513,1473905394,1473908406,1
1168514,104892989,1549506903,1


In [10]:
# import pandas as pd
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import precision_recall_curve, auc
# import joblib
# import json

# # Функция для загрузки данных
# def load_data(sample_size=0.1):
#     # Загрузка данных из файлов Parquet
#     attributes = pd.read_parquet('F:/competition/attributes.parquet')
#     resnet = pd.read_parquet('F:/competition/resnet.parquet')
#     text_and_bert = pd.read_parquet('F:/competition/text_and_bert.parquet')
#     train = pd.read_parquet('F:/competition/train.parquet')
#     test = pd.read_parquet('F:/competition/test.parquet')

#     # Если sample_size меньше 1, то выборка данных
#     if sample_size < 1:
#         attributes = attributes.sample(frac=sample_size, random_state=42)
#         resnet = resnet.sample(frac=sample_size, random_state=42)
#         text_and_bert = text_and_bert.sample(frac=sample_size, random_state=42)
#         train = train.sample(frac=sample_size, random_state=42)
#         test = test.sample(frac=sample_size, random_state=42)

#     return attributes, resnet, text_and_bert, train, test
# # Шаг 1: Загрузка данных и проверка наличия столбцов
# def load_parquet_file(path, expected_columns):
#     """Функция для загрузки данных из файла Parquet с проверкой столбцов."""
#     try:
#         df = pd.read_parquet(path)
#         missing_cols = [col for col in expected_columns if col not in df.columns]
#         if missing_cols:
#             raise ValueError(f"Отсутствуют столбцы в файле {path}: {', '.join(missing_cols)}")
#         return df
#     except Exception as e:
#         print(f"Ошибка при загрузке {path}: {e}")
#         return pd.DataFrame()  # Возвращаем пустой DataFrame в случае ошибки

# # Загрузка данных из файлов с проверкой столбцов
# attributes = load_parquet_file(attributes_path, ['variantid', 'categories', 'characteristic_attributes_mapping'])
# resnet = load_parquet_file(resnet_path, ['variantid', 'main_pic_embeddings_resnet_v1'])
# text_and_bert = load_parquet_file(text_and_bert_path, ['variantid', 'combined_text'])
# train = load_parquet_file(train_path, ['variantid1', 'variantid2', 'target'])

# # Шаг 2: Обработка атрибутов и текстов
# if not attributes.empty:
#     # Преобразование JSON строк в объекты
#     attributes['categories'] = attributes['categories'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)
#     attributes['characteristic_attributes_mapping'] = attributes['characteristic_attributes_mapping'].apply(lambda x: json.loads(x) if isinstance(x, str) else x)

#     # Функция для извлечения текста из строки
#     def extract_text_from_row(row):
#         category_text = ' '.join(
#             [' '.join(map(str, v)) if isinstance(v, list) else str(v) for v in list(row['categories'].values())]
#         )
#         attributes_text = ' '.join(
#             [' '.join(map(str, v)) if isinstance(v, list) else str(v) for v in list(row['characteristic_attributes_mapping'].values())]
#         )
#         return f"{category_text} {attributes_text}"

#     # Применение функции к каждому ряду данных
#     attributes['combined_text'] = attributes.apply(extract_text_from_row, axis=1)

# # Проверка наличия необходимых столбцов перед объединением
# if not text_and_bert.empty and 'combined_text' in text_and_bert.columns:
#     combined_df = attributes.merge(text_and_bert[['variantid', 'combined_text']], on='variantid', how='left')
# else:
#     combined_df = attributes

# # Шаг 3: Объединение данных
# if not train.empty and not resnet.empty and not combined_df.empty:
#     train_data = train.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid1', right_on='variantid', how='left')
#     train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_1'})
#     train_data = train_data.drop(columns=['variantid'])

#     train_data = train_data.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid2', right_on='variantid', how='left')
#     train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_2'})
#     train_data = train_data.drop(columns=['variantid'])

#     train_data = train_data.merge(combined_df[['variantid', 'combined_text']], left_on='variantid1', right_on='variantid', how='left')
#     train_data = train_data.rename(columns={'combined_text': 'text_1'})
#     train_data = train_data.drop(columns=['variantid'])

#     train_data = train_data.merge(combined_df[['variantid', 'combined_text']], left_on='variantid2', right_on='variantid', how='left')
#     train_data = train_data.rename(columns={'combined_text': 'text_2'})
#     train_data = train_data.drop(columns=['variantid'])

#     train_data = train_data.dropna()  # Удаление строк с пропущенными значениями
# else:
#     train_data = pd.DataFrame()

# # Шаг 4: Подготовка данных
if not train_data.empty:
    # Создание объекта TfidfVectorizer
    tfidf_vectorizer = TfidfVectorizer(max_features=2000)

    # Преобразование текстовых данных в эмбеддинги
    text_data = train_data['text_1'].fillna('') + ' ' + train_data['text_2'].fillna('')
    text_embeddings = tfidf_vectorizer.fit_transform(text_data).toarray()

    # Функция для объединения эмбеддингов изображений и текста
    def combine_embeddings(row):
        pic_embeddings = np.concatenate([row['pic_embeddings_1'][0], row['pic_embeddings_2'][0]])
        text_embeddings_row = text_embeddings[row.name]
        return np.concatenate([pic_embeddings, text_embeddings_row])

    # Применение функции к данным
    train_data['combined_embeddings'] = train_data.apply(combine_embeddings, axis=1)

    # Создание матрицы признаков и вектора целей
    X = np.vstack(train_data['combined_embeddings'].values)
    y = train_data['target']

    # Разделение данных на обучающую и валидационную выборки
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=23)
else:
    X_train, X_val, y_train, y_val = (np.array([]),) * 4

# Шаг 5: Обучение модели
if X_train.size > 0 and y_train.size > 0:
    # Создание и обучение модели логистической регрессии
    model = LogisticRegression(max_iter=2000)
    model.fit(X_train, y_train)

    # Сохранение модели
    joblib.dump(model, 'baseline3.pkl')

# Шаг 6: Оценка модели
if X_val.size > 0 and y_val.size > 0:
    # Прогнозирование вероятностей
    y_pred_prob = model.predict_proba(X_val)[:, 1]
    y_pred = (y_pred_prob >= 0.5).astype(int)

    # Вычисление метрик точности и полноты
    precision, recall, _ = precision_recall_curve(y_val, y_pred_prob)
    prauc = auc(recall, precision)
    print(f'PRAUC: {prauc}')

    # Сохранение вектораизатора
    joblib.dump(tfidf_vectorizer, 'vectorizer3.pkl')


PRAUC: 0.7724866290617343


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc
import joblib
import json

# Функция для загрузки данных
def load_data(sample_size=0.1):
    # Загрузка данных из файлов Parquet
    attributes = pd.read_parquet('F:/competition/attributes.parquet')
    resnet = pd.read_parquet('F:/competition/resnet.parquet')
    text_and_bert = pd.read_parquet('F:/competition/text_and_bert.parquet')
    train = pd.read_parquet('F:/competition/train.parquet')
    test = pd.read_parquet('F:/competition/test.parquet')

    # Если sample_size меньше 1, то выборка данных
    if sample_size < 1:
        attributes = attributes.sample(frac=sample_size, random_state=42)
        resnet = resnet.sample(frac=sample_size, random_state=42)
        text_and_bert = text_and_bert.sample(frac=sample_size, random_state=42)
        train = train.sample(frac=sample_size, random_state=42)
        test = test.sample(frac=sample_size, random_state=42)

    return attributes, resnet, text_and_bert, train, test

In [3]:
attributes, resnet, text_and_bert, train, test = load_data()

In [8]:
# Просмотр первых нескольких строк каждого DataFrame
print("Attributes DataFrame:")
attributes.head(3)  # Вывод первых 5 строк DataFrame

Attributes DataFrame:


,variantid,categories,characteristic_attributes_mapping
809556,1068398160,"{""1"": ""Одежда и обувь"", ""2"": ""Одежда"", ""3"": ""О...","{""Модель"": [""вязаная"", ""утепленная модель""], ""..."
253664,80256086,"{""1"": ""EPG"", ""2"": ""Строительство и ремонт"", ""3...","{""Комплектация"": [""светильник, инструкция""], ""..."
214869,1402305170,"{""1"": ""EPG"", ""2"": ""Строительство и ремонт"", ""3...","{""Комплектация"": [""Встраиваемый светильник Art..."


In [9]:
# Просмотр первых нескольких строк каждого DataFrame
print("\nResnet DataFrame:")
resnet.head(3)


Resnet DataFrame:


,variantid,main_pic_embeddings_resnet_v1,pic_embeddings_resnet_v1
809556,1068398160,"[[0.4887434244155884, 0.2201225757598877, -0.2...","[[0.49651038646698, 0.3525044620037079, -0.650..."
253664,80256086,"[[0.15673470497131348, 0.19669684767723083, 1....","[[0.22057659924030304, 0.23254850506782532, 0...."
214869,1402305170,"[[-1.1406502723693848, 0.29915758967399597, -0...","[[-0.22224298119544983, 0.037943556904792786, ..."


In [10]:
print("\nText and Bert DataFrame:")
text_and_bert.head(3)



Text and Bert DataFrame:


,variantid,name,description,name_bert_64
809556,1068398160,Пинетки RUSKNIT Малышам,Долгожданная новинка! Утепленные пинетки вязан...,"[-0.4336099624633789, 0.33932164311408997, 0.7..."
253664,80256086,Подвесной светильник Maytoni Uva MOD059PL-03G,Светильник потолочный подвесной Maytoni MOD059...,"[-0.5351067781448364, 0.6248574256896973, 0.52..."
214869,1402305170,Arte Lamp Встраиваемый светильник,Встраиваемый светильник Arte Lamp Keid A2162PL...,"[-0.34187906980514526, 0.4898632764816284, 0.7..."


In [11]:
print("\nTrain DataFrame:")
train.head(3)


Train DataFrame:


,variantid1,variantid2,target
181389,938776155,473815138,1
269066,1349745192,1349703019,0
1044282,1450853415,1450853943,1


In [12]:
print("\nTest DataFrame:")
test.head(3)


Test DataFrame:


,variantid1,variantid2
12928,641921785,646752712
17577,1735144642,1727990945
25225,1785914688,1769974301


In [13]:
# Для более подробного анализа можно также использовать метод .info() для отображения информации о столбцах и типах данных
print("\nAttributes DataFrame Info:")
attributes.info()

print("\nResnet DataFrame Info:")
resnet.info()

print("\nText and Bert DataFrame Info:")
text_and_bert.info()

print("\nTrain DataFrame Info:")
train.info()



Attributes DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 225257 entries, 809556 to 2179262
Data columns (total 3 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   variantid                          225257 non-null  int64 
 1   categories                         225257 non-null  object
 2   characteristic_attributes_mapping  225257 non-null  object
dtypes: int64(1), object(2)
memory usage: 6.9+ MB

Resnet DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 225257 entries, 809556 to 2179262
Data columns (total 3 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   variantid                      225257 non-null  int64 
 1   main_pic_embeddings_resnet_v1  225257 non-null  object
 2   pic_embeddings_resnet_v1       166276 non-null  object
dtypes: int64(1), object(2)
memory usage: 6.9+ MB

Text 

# Предобработка таблицы Attributes

In [7]:
import pandas as pd
import json

# Загрузка данных
#attributes = pd.read_parquet('F:/competition/attributes.parquet')

# Преобразование JSON-строк в словари Python
attributes['categories'] = attributes['categories'].apply(json.loads)
attributes['characteristic_attributes_mapping'] = attributes['characteristic_attributes_mapping'].apply(json.loads)

# Пример проверки преобразованных данных
attributes.head(3)


TypeError: the JSON object must be str, bytes or bytearray, not dict

In [17]:
attributes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 225257 entries, 809556 to 2179262
Data columns (total 3 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   variantid                          225257 non-null  int64 
 1   categories                         225257 non-null  object
 2   characteristic_attributes_mapping  225257 non-null  object
dtypes: int64(1), object(2)
memory usage: 6.9+ MB


# Предобработка таблицы Resnet

In [22]:
import pandas as pd
import numpy as np

# Загрузка данных
#resnet = pd.read_parquet('F:/competition/resnet.parquet')

# Проверка формата данных в столбцах
print(resnet.dtypes)
print(resnet.head())

# Преобразование данных в массивы numpy (если они не уже в таком формате)
def ensure_array(data):
    if isinstance(data, str):  # Если данные строка, предположительно в формате JSON
        try:
            return np.array(json.loads(data))
        except json.JSONDecodeError:
            return np.zeros(0)
    elif isinstance(data, (list, np.ndarray)):  # Если данные уже список или массив
        return np.array(data)
    else:
        return np.zeros(0)

# Преобразование строк JSON в numpy массивы (если необходимо)
resnet['main_pic_embeddings_resnet_v1'] = resnet['main_pic_embeddings_resnet_v1'].apply(ensure_array)
resnet['pic_embeddings_resnet_v1'] = resnet['pic_embeddings_resnet_v1'].apply(ensure_array)

# Пример проверки преобразованных данных
resnet.head(3)


variantid                         int64
main_pic_embeddings_resnet_v1    object
pic_embeddings_resnet_v1         object
dtype: object


,variantid,main_pic_embeddings_resnet_v1,pic_embeddings_resnet_v1
809556,1068398160,"[[0.4887434244155884, 0.2201225757598877, -0.2...","[[0.49651038646698, 0.3525044620037079, -0.650..."
253664,80256086,"[[0.15673470497131348, 0.19669684767723083, 1....","[[0.22057659924030304, 0.23254850506782532, 0...."
214869,1402305170,"[[-1.1406502723693848, 0.29915758967399597, -0...","[[-0.22224298119544983, 0.037943556904792786, ..."


In [19]:
resnet.dtypes

variantid                         int64
main_pic_embeddings_resnet_v1    object
pic_embeddings_resnet_v1         object
dtype: object

# Предобработка таблицы Text and Bert

In [16]:
import pandas as pd
import numpy as np
import json

# Загрузка данных
#text_and_bert = pd.read_parquet('F:/competition/text_and_bert.parquet')

# Преобразование строк JSON в numpy массивы
def ensure_array(data):
    if isinstance(data, str):  # Если данные строка, предположительно в формате JSON
        try:
            return np.array(json.loads(data))
        except json.JSONDecodeError:
            return np.zeros(0)
    elif isinstance(data, (list, np.ndarray)):  # Если данные уже список или массив
        return np.array(data)
    else:
        return np.zeros(0)

text_and_bert['name_bert_64'] = text_and_bert['name_bert_64'].apply(ensure_array)

# Пример проверки преобразованных данных
text_and_bert.head(3)


,variantid,name,description,name_bert_64
809556,1068398160,Пинетки RUSKNIT Малышам,Долгожданная новинка! Утепленные пинетки вязан...,"[-0.4336099624633789, 0.33932164311408997, 0.7..."
253664,80256086,Подвесной светильник Maytoni Uva MOD059PL-03G,Светильник потолочный подвесной Maytoni MOD059...,"[-0.5351067781448364, 0.6248574256896973, 0.52..."
214869,1402305170,Arte Lamp Встраиваемый светильник,Встраиваемый светильник Arte Lamp Keid A2162PL...,"[-0.34187906980514526, 0.4898632764816284, 0.7..."


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc
import joblib
import json

# Функция для загрузки данных
def load_data(sample_size=1):
    # Загрузка данных из файлов Parquet
    attributes = pd.read_parquet('F:/competition/attributes.parquet')
    resnet = pd.read_parquet('F:/competition/resnet.parquet')
    text_and_bert = pd.read_parquet('F:/competition/text_and_bert.parquet')
    train = pd.read_parquet('F:/competition/train.parquet')
    test = pd.read_parquet('F:/competition/test.parquet')

    # Если sample_size меньше 1, то выборка данных
    if sample_size < 1:
        attributes = attributes.sample(frac=sample_size, random_state=42)
        resnet = resnet.sample(frac=sample_size, random_state=42)
        text_and_bert = text_and_bert.sample(frac=sample_size, random_state=42)
        train = train.sample(frac=sample_size, random_state=42)
        test = test.sample(frac=sample_size, random_state=42)

    return attributes, resnet, text_and_bert, train, test

In [2]:
attributes, resnet, text_and_bert, train, test = load_data(sample_size=0.1) 

In [5]:
attributes.head(3)

,variantid,categories,characteristic_attributes_mapping
809556,1068398160,"{'1': 'Одежда и обувь', '2': 'Одежда', '3': 'О...","{'Модель': ['вязаная', 'утепленная модель'], '..."
253664,80256086,"{'1': 'EPG', '2': 'Строительство и ремонт', '3...","{'Комплектация': ['светильник, инструкция'], '..."
214869,1402305170,"{'1': 'EPG', '2': 'Строительство и ремонт', '3...",{'Комплектация': ['Встраиваемый светильник Art...


In [6]:
# Преобразование строк JSON в объекты Python
attributes['categories'] = attributes['categories'].apply(json.loads)
attributes['characteristic_attributes_mapping'] = attributes['characteristic_attributes_mapping'].apply(json.loads)

TypeError: the JSON object must be str, bytes or bytearray, not dict

In [ ]:


# # Заполнение пропущенных значений в 'description' пустыми строками
# text_and_bert['description'] = text_and_bert['description'].fillna('')

# # Объединение таблиц
# # Объединение с Resnet для pic_embeddings_1
# train_data = train.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid1', right_on='variantid', how='left')
# train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_1'})
# train_data = train_data.drop(columns=['variantid'])

# # Объединение с Resnet для pic_embeddings_2
# train_data = train_data.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid2', right_on='variantid', how='left')
# train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_2'})
# train_data = train_data.drop(columns=['variantid'])

# # Объединение с Text and Bert для text_embedding_1
# train_data = train_data.merge(text_and_bert[['variantid', 'name_bert_64']], left_on='variantid1', right_on='variantid', how='left')
# train_data = train_data.rename(columns={'name_bert_64': 'text_embedding_1'})
# train_data = train_data.drop(columns=['variantid'])

# # Объединение с Text and Bert для text_embedding_2
# train_data = train_data.merge(text_and_bert[['variantid', 'name_bert_64']], left_on='variantid2', right_on='variantid', how='left')
# train_data = train_data.rename(columns={'name_bert_64': 'text_embedding_2'})
# train_data = train_data.drop(columns=['variantid'])

# # Удаление строк с пропущенными значениями
# train_data = train_data.dropna()

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc
import joblib
import json

# # Функция для загрузки данных
# def load_data(sample_size=0.1):
#     # Загрузка данных из файлов Parquet
#     attributes = pd.read_parquet('F:/competition/attributes.parquet')
#     resnet = pd.read_parquet('F:/competition/resnet.parquet')
#     text_and_bert = pd.read_parquet('F:/competition/text_and_bert.parquet')
#     train = pd.read_parquet('F:/competition/train.parquet')
#     test = pd.read_parquet('F:/competition/test.parquet')

#     # Если sample_size меньше 1, то выборка данных
#     if sample_size < 1:
#         attributes = attributes.sample(frac=sample_size, random_state=42)
#         resnet = resnet.sample(frac=sample_size, random_state=42)
#         text_and_bert = text_and_bert.sample(frac=sample_size, random_state=42)
#         train = train.sample(frac=sample_size, random_state=42)
#         test = test.sample(frac=sample_size, random_state=42)

#     return attributes, resnet, text_and_bert, train, test

# Функция для обработки данных
def process_data(attributes, resnet, text_and_bert, train):
    # Преобразование строк JSON в объекты Python
    attributes['categories'] = attributes['categories'].apply(json.loads)
    attributes['characteristic_attributes_mapping'] = attributes['characteristic_attributes_mapping'].apply(json.loads)
    
    # Заполнение пропущенных значений в 'description' пустыми строками
    text_and_bert['description'] = text_and_bert['description'].fillna('')

    # Объединение таблиц
    # Объединение с Resnet для pic_embeddings_1
    train_data = train.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid1', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_1'})
    train_data = train_data.drop(columns=['variantid'])

    # Объединение с Resnet для pic_embeddings_2
    train_data = train_data.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid2', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_2'})
    train_data = train_data.drop(columns=['variantid'])

    # Объединение с Text and Bert для text_embedding_1
    train_data = train_data.merge(text_and_bert[['variantid', 'name_bert_64']], left_on='variantid1', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'name_bert_64': 'text_embedding_1'})
    train_data = train_data.drop(columns=['variantid'])

    # Объединение с Text and Bert для text_embedding_2
    train_data = train_data.merge(text_and_bert[['variantid', 'name_bert_64']], left_on='variantid2', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'name_bert_64': 'text_embedding_2'})
    train_data = train_data.drop(columns=['variantid'])

    # Удаление строк с пропущенными значениями
    train_data = train_data.dropna()

    return train_data

# Функция для комбинирования эмбеддингов
def combine_embeddings(row):
    try:
        # Преобразование эмбеддингов из строковых представлений в массивы NumPy
        pic_embeddings_1 = np.array(row['pic_embeddings_1'])
        pic_embeddings_2 = np.array(row['pic_embeddings_2'])
        text_embeddings_1 = np.array(row['text_embedding_1'])
        text_embeddings_2 = np.array(row['text_embedding_2'])

        # Проверка на пустые эмбеддинги
        if pic_embeddings_1.size == 0 or pic_embeddings_2.size == 0 or text_embeddings_1.size == 0 or text_embeddings_2.size == 0:
            return np.zeros((0,))

        # Проверка на согласованность размеров
        if len(pic_embeddings_1) != len(pic_embeddings_2) or len(text_embeddings_1) != len(text_embeddings_2):
            return np.zeros((0,))

        # Объединение эмбеддингов
        pic_embeddings = np.concatenate([pic_embeddings_1, pic_embeddings_2])
        text_embeddings = np.concatenate([text_embeddings_1, text_embeddings_2])

        return np.concatenate([pic_embeddings, text_embeddings])
    
    except Exception as e:
        print(f"Ошибка преобразования эмбеддингов: {e}")
        return np.zeros((0,))

# Функция для подготовки данных для обучения модели
def prepare_data(train_data):
    # Создание комбинированных эмбеддингов
    train_data['combined_embeddings'] = train_data.apply(combine_embeddings, axis=1)

    # Отделение эмбеддингов от целевой переменной
    combined_embeddings = train_data['combined_embeddings'].tolist()

    # Удаление строк с пустыми эмбеддингами
    valid_rows = [embeddings for embeddings in combined_embeddings if embeddings.size > 0]
    if len(valid_rows) == 0:
        raise ValueError("Все строки содержат пустые эмбеддинги.")
    
    # Преобразование списка валидных эмбеддингов в массив NumPy
    X = np.vstack(valid_rows)
    y = train_data.loc[train_data['combined_embeddings'].apply(lambda x: x.size > 0), 'target'].values

    # Разделение данных на обучающую и валидационную выборки
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=23)

    return X_train, X_val, y_train, y_val

# Функция для обучения модели
def train_model(X_train, y_train):
    # Создание и обучение модели логистической регрессии
    model = LogisticRegression(max_iter=2000)
    model.fit(X_train, y_train)
    
    # Сохранение модели на диск
    joblib.dump(model, 'model.pkl')
    return model

# Функция для оценки модели
def evaluate_model(model, X_val, y_val):
    # Получение вероятностей предсказания
    y_pred_prob = model.predict_proba(X_val)[:, 1]
    
    # Вычисление Precision-Recall AUC
    precision, recall, _ = precision_recall_curve(y_val, y_pred_prob)
    pr_auc = auc(recall, precision)
    print(f'PR AUC: {pr_auc}')

# Основная функция
def main():
    # Загрузка данных
    #attributes, resnet, text_and_bert, train, test = load_data(sample_size=0.1)
    
    # Обработка данных
    train_data = process_data(attributes, resnet, text_and_bert, train)
    
    # Подготовка данных для обучения
    X_train, X_val, y_train, y_val = prepare_data(train_data)

    # Обучение модели
    model = train_model(X_train, y_train)
    
    # Оценка модели
    evaluate_model(model, X_val, y_val)

    # Подготовка данных для тестирования
    # test_data = process_test_data(test, resnet, text_and_bert)  # Нужно реализовать эту функцию
    # test_predictions = model.predict_proba(test_data)[:, 1]
    # Создание файла с результатами
    # submission = pd.DataFrame({'id': test['id'], 'target': test_predictions})
    # submission.to_csv('submission.csv', index=False)

# Запуск основной функции, если скрипт выполняется напрямую
if __name__ == "__main__":
    main()


ValueError: setting an array element with a sequence.

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc
import joblib
import json

# Функция для обработки данных
def process_data(attributes, resnet, text_and_bert, train):
    # Проверка типа данных в столбцах 'categories' и 'characteristic_attributes_mapping'
    if isinstance(attributes['categories'].iloc[0], str):
        attributes['categories'] = attributes['categories'].apply(json.loads)
    if isinstance(attributes['characteristic_attributes_mapping'].iloc[0], str):
        attributes['characteristic_attributes_mapping'] = attributes['characteristic_attributes_mapping'].apply(json.loads)
    
    # Заполнение пропущенных значений в 'description' пустыми строками
    text_and_bert['description'] = text_and_bert['description'].fillna('')

    # Объединение таблиц
    train_data = train.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid1', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_1'})
    train_data = train_data.drop(columns=['variantid'])

    train_data = train_data.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid2', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_2'})
    train_data = train_data.drop(columns=['variantid'])

    train_data = train_data.merge(text_and_bert[['variantid', 'name_bert_64']], left_on='variantid1', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'name_bert_64': 'text_embedding_1'})
    train_data = train_data.drop(columns=['variantid'])

    train_data = train_data.merge(text_and_bert[['variantid', 'name_bert_64']], left_on='variantid2', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'name_bert_64': 'text_embedding_2'})
    train_data = train_data.drop(columns=['variantid'])

    # Удаление строк с пропущенными значениями
    train_data = train_data.dropna()

    return train_data

# Функция для подготовки данных для обучения модели
def prepare_data(train_data):
    # Разворачиваем векторы эмбеддингов в одномерные массивы
    train_data['pic_embeddings_1'] = train_data['pic_embeddings_1'].apply(lambda x: np.array(x).flatten())
    train_data['pic_embeddings_2'] = train_data['pic_embeddings_2'].apply(lambda x: np.array(x).flatten())
    train_data['text_embedding_1'] = train_data['text_embedding_1'].apply(lambda x: np.array(x).flatten())
    train_data['text_embedding_2'] = train_data['text_embedding_2'].apply(lambda x: np.array(x).flatten())

    # Объединение всех признаков в один массив
    X = np.hstack([
        np.stack(train_data['pic_embeddings_1']),
        np.stack(train_data['pic_embeddings_2']),
        np.stack(train_data['text_embedding_1']),
        np.stack(train_data['text_embedding_2'])
    ])

    y = train_data['target'].values
    return X, y

# Функция для обучения модели
def train_model(X_train, y_train):
    # Создание и обучение модели логистической регрессии
    model = LogisticRegression(max_iter=2000)
    model.fit(X_train, y_train)
    
    # Сохранение модели на диск
    joblib.dump(model, 'model.pkl')
    return model

# Функция для оценки модели
def evaluate_model(model, X_val, y_val):
    # Получение вероятностей предсказания
    y_pred_prob = model.predict_proba(X_val)[:, 1]
    
    # Вычисление Precision-Recall AUC
    precision, recall, _ = precision_recall_curve(y_val, y_pred_prob)
    pr_auc = auc(recall, precision)
    print(f'PR AUC: {pr_auc}')

# Основная функция
def main():
    # Загрузка данных
    # attributes, resnet, text_and_bert, train, test = load_data(sample_size=0.1)
    
    # Обработка данных
    train_data = process_data(attributes, resnet, text_and_bert, train)
    
    # Подготовка данных для обучения
    X, y = prepare_data(train_data)
    
    # Разделение данных на обучающую и валидационную выборки
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=23)

    # Обучение модели
    model = train_model(X_train, y_train)
    
    # Оценка модели
    evaluate_model(model, X_val, y_val)

    # Подготовка данных для тестирования
    # test_data = process_test_data(test, resnet, text_and_bert)  # Нужно реализовать эту функцию
    # test_predictions = model.predict_proba(test_data)[:, 1]
    # Создание файла с результатами
    # submission = pd.DataFrame({'id': test['id'], 'target': test_predictions})
    # submission.to_csv('submission.csv', index=False)

# Запуск основной функции, если скрипт выполняется напрямую
if __name__ == "__main__":
    main()


ValueError: setting an array element with a sequence.

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc
import joblib
import json

# # Функция для загрузки данных
# def load_data(sample_size=0.1):
#     # Загрузка данных из файлов Parquet
#     attributes = pd.read_parquet('F:/competition/attributes.parquet')
#     resnet = pd.read_parquet('F:/competition/resnet.parquet')
#     text_and_bert = pd.read_parquet('F:/competition/text_and_bert.parquet')
#     train = pd.read_parquet('F:/competition/train.parquet')
#     test = pd.read_parquet('F:/competition/test.parquet')

#     # Если sample_size меньше 1, то выборка данных
#     if sample_size < 1:
#         attributes = attributes.sample(frac=sample_size, random_state=42)
#         resnet = resnet.sample(frac=sample_size, random_state=42)
#         text_and_bert = text_and_bert.sample(frac=sample_size, random_state=42)
#         train = train.sample(frac=sample_size, random_state=42)
#         test = test.sample(frac=sample_size, random_state=42)

#     return attributes, resnet, text_and_bert, train, test

# Функция для обработки данных
def process_data(attributes, resnet, text_and_bert, train):
    # Преобразование строк JSON в объекты Python
    attributes['categories'] = attributes['categories'].apply(json.loads)
    attributes['characteristic_attributes_mapping'] = attributes['characteristic_attributes_mapping'].apply(json.loads)
    
    # Заполнение пропущенных значений в 'description' пустыми строками
    text_and_bert['description'] = text_and_bert['description'].fillna('')

    # Объединение таблиц
    # Объединение с Resnet для pic_embeddings_1
    train_data = train.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid1', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_1'})
    train_data = train_data.drop(columns=['variantid'])

    # Объединение с Resnet для pic_embeddings_2
    train_data = train_data.merge(resnet[['variantid', 'main_pic_embeddings_resnet_v1']], left_on='variantid2', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'main_pic_embeddings_resnet_v1': 'pic_embeddings_2'})
    train_data = train_data.drop(columns=['variantid'])

    # Объединение с Text and Bert для text_embedding_1
    train_data = train_data.merge(text_and_bert[['variantid', 'name_bert_64']], left_on='variantid1', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'name_bert_64': 'text_embedding_1'})
    train_data = train_data.drop(columns=['variantid'])

    # Объединение с Text and Bert для text_embedding_2
    train_data = train_data.merge(text_and_bert[['variantid', 'name_bert_64']], left_on='variantid2', right_on='variantid', how='left')
    train_data = train_data.rename(columns={'name_bert_64': 'text_embedding_2'})
    train_data = train_data.drop(columns=['variantid'])

    # Удаление строк с пропущенными значениями
    train_data = train_data.dropna()

    return train_data

# Функция для комбинирования эмбеддингов
def combine_embeddings(row):
    try:
        # Преобразование эмбеддингов из строковых представлений в массивы NumPy
        pic_embeddings_1 = np.array(row['pic_embeddings_1'])
        pic_embeddings_2 = np.array(row['pic_embeddings_2'])
        text_embeddings_1 = np.array(row['text_embedding_1'])
        text_embeddings_2 = np.array(row['text_embedding_2'])

        # Проверка на пустые эмбеддинги
        if pic_embeddings_1.size == 0 or pic_embeddings_2.size == 0 or text_embeddings_1.size == 0 or text_embeddings_2.size == 0:
            return np.zeros((0,))

        # Проверка на согласованность размеров
        if len(pic_embeddings_1) != len(pic_embeddings_2) or len(text_embeddings_1) != len(text_embeddings_2):
            return np.zeros((0,))

        # Объединение эмбеддингов
        pic_embeddings = np.concatenate([pic_embeddings_1, pic_embeddings_2])
        text_embeddings = np.concatenate([text_embeddings_1, text_embeddings_2])

        return np.concatenate([pic_embeddings, text_embeddings])
    
    except Exception as e:
        print(f"Ошибка преобразования эмбеддингов: {e}")
        return np.zeros((0,))

# Функция для подготовки данных для обучения модели
def prepare_data(train_data):
    # Создание комбинированных эмбеддингов
    train_data['combined_embeddings'] = train_data.apply(combine_embeddings, axis=1)

    # Отделение эмбеддингов от целевой переменной
    combined_embeddings = train_data['combined_embeddings'].tolist()

    # Удаление строк с пустыми эмбеддингами
    valid_rows = [embeddings for embeddings in combined_embeddings if embeddings.size > 0]
    if len(valid_rows) == 0:
        raise ValueError("Все строки содержат пустые эмбеддинги.")
    
    # Преобразование списка валидных эмбеддингов в массив NumPy
    X = np.vstack(valid_rows)
    y = train_data.loc[train_data['combined_embeddings'].apply(lambda x: x.size > 0), 'target'].values

    # Убедитесь, что X и y имеют согласованную длину
    if X.shape[0] != len(y):
        raise ValueError("Размеры X и y не совпадают.")

    # Разделение данных на обучающую и валидационную выборки
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=23)

    return X_train, X_val, y_train, y_val

# Функция для обучения модели
def train_model(X_train, y_train):
    # Проверка размерностей перед обучением
    print(f"Размер X_train: {X_train.shape}")
    print(f"Размер y_train: {len(y_train)}")

    # Создание и обучение модели логистической регрессии
    model = LogisticRegression(max_iter=2000)
    model.fit(X_train, y_train)
    
    # Сохранение модели на диск
    joblib.dump(model, 'model.pkl')
    return model

# Функция для оценки модели
def evaluate_model(model, X_val, y_val):
    # Получение вероятностей предсказания
    y_pred_prob = model.predict_proba(X_val)[:, 1]
    
    # Вычисление Precision-Recall AUC
    precision, recall, _ = precision_recall_curve(y_val, y_pred_prob)
    pr_auc = auc(recall, precision)
    print(f'PR AUC: {pr_auc}')

# Основная функция
def main():
    # Загрузка данных
    attributes, resnet, text_and_bert, train, test = load_data(sample_size=0.1)
    
    # Обработка данных
    train_data = process_data(attributes, resnet, text_and_bert, train)
    
    # Подготовка данных для обучения
    X_train, X_val, y_train, y_val = prepare_data(train_data)

    # Обучение модели
    model = train_model(X_train, y_train)
    
    # Оценка модели
    evaluate_model(model, X_val, y_val)

    # Подготовка данных для тестирования
    # test_data = process_test_data(test, resnet, text_and_bert)  # Нужно реализовать эту функцию
    # test_predictions = model.predict_proba(test_data)[:, 1]
    # Создание файла с результатами
    # submission = pd.DataFrame({'id': test['id'], 'target': test_predictions})
    # submission.to_csv('submission.csv', index=False)

# Запуск основной функции, если скрипт выполняется напрямую
if __name__ == "__main__":
    main()


Размер X_train: (933, 130)
Размер y_train: 933


ValueError: setting an array element with a sequence.